In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import sys
from itertools import product

In [1]:
from hypeflow.hype import hypeflow

ImportError: cannot import name 'hypeflow' from 'hypeflow.hype' (/home/shg096/hypeflow/src/hypeflow/hype.py)

In [23]:
# load the information from the gistool for soil and land cover and find the number of geoclass

soil_type = pd.read_csv('../data/gistool_outputs/soil_classes/Bow_at_Calgary_stats_soil_classes.csv')
landcover_type = pd.read_csv('../data/gistool_outputs/landsat/Bow_at_Calgary_stats_NA_NALCMS_landcover_2020_30m.csv')
elevation_mean = pd.read_csv('../data/gistool_outputs/merit_hydro/Bow_at_Calgary_stats_elv.csv')

soil_type = soil_type.sort_values(by='COMID').reset_index(drop=True)
landcover_type = landcover_type.sort_values(by='COMID').reset_index(drop=True)
elevation_mean = elevation_mean.sort_values(by='COMID').reset_index(drop=True)


In [24]:
# check if COMIDs are the similar
# check the len
if len(soil_type) != len(landcover_type):
    sys.exit('The provided length of soil and land cover is not identical')

# check if the COMIDs are similar
if sum(soil_type['COMID'].values - landcover_type['COMID'].values) != 0:
    sys.exit('The COMID of the shapefile in soil and land cover is not identical')
    

In [25]:
# Find the majority value of the "majority" column
majority_value = soil_type['majority'].mode().values[0]

# Replace 0 values with the majority value
soil_type['majority'] = soil_type['majority'].replace(0, majority_value)

soil_type

,COMID,majority
0,71022153,3
1,71022160,3
2,71022164,3
3,71022165,3
4,71022168,3
...,...,...
448,71039048,3
449,71039109,3
450,71039121,3
451,71039250,3


In [26]:
# Possible normalization of land cover fraction above a certain threshold
# First iteration: Set values less than 0.05 to 0
for col in landcover_type.columns:
    if col.startswith('frac_'):
        landcover_type[col] = landcover_type[col].apply(lambda x: 0 if x < 0.0 else x)

# Second iteration: Normalize non-zero values based on row sums
for index, row in landcover_type.iterrows():
    frac_columns = [col for col in landcover_type.columns if col.startswith('frac_')]
    row_sum = row[frac_columns].sum()
    if row_sum > 0:
        for col in frac_columns:
            landcover_type.at[index, col] /= row_sum
    


In [27]:
# find the combination of the majority soil and land cover
combinations_set_all = set()
for index, row in landcover_type.iterrows():
    # get the fraction for land cover for each row
    fractions = [col for col in landcover_type.columns if col.startswith('frac') and row[col] > 0.00]
    # remove frac_ from the list
    fractions = [col.split('_')[1] for col in fractions]
    fractions = [int(name) for name in fractions]

    # get the majority soil type for each row
    majority_soil = [soil_type['majority'].iloc[index].item()]

    # Combine as combination of soil and land cover and keep as a set
    combinations = list(product(fractions, majority_soil))
    combinations_set = set(combinations)
    combinations_set_all.update(combinations_set)

print(combinations_set_all)
print(len(combinations_set_all))
    

{(8, 8), (19, 8), (10, 8), (16, 3), (1, 8), (17, 3), (18, 3), (8, 3), (19, 3), (10, 3), (1, 3), (6, 8), (2, 3), (14, 3), (15, 3), (5, 3), (16, 8), (6, 3), (18, 8)}
19


In [28]:
data_list = [{'landcover': item[0], 'soil': item[1]} for item in combinations_set_all]

# Create a pandas DataFrame from the list of dictionaries
combination = pd.DataFrame(data_list)

combination ['SLC'] = 0
combination ['SLC'] = np.arange(len(combination))+1

combination

,landcover,soil,SLC
0,8,8,1
1,19,8,2
2,10,8,3
3,16,3,4
4,1,8,5
5,17,3,6
6,18,3,7
7,8,3,8
8,19,3,9
9,10,3,10


In [29]:
landcover_type_prepared = landcover_type.copy()



for i in range(1, len(combination)+1):
    column_name = f'SLC_{i}'
    landcover_type_prepared[column_name] = 0.00

landcover_type_prepared['soil'] = soil_type['majority']

def get_non_zero_columns(row):
    return [col for col in row.index if col.startswith('frac_') and row[col] != 0]

# Apply the function to each row
landcover_type_prepared['non_zero_columns'] = landcover_type_prepared.apply(get_non_zero_columns, axis=1)


for index, row in landcover_type_prepared.iterrows():
    # get the soil type
    soil_type_value = soil_type['majority'].iloc[index]

    for i in row['non_zero_columns']:

        # remove frac from column name 
        land_cover_value = i.replace("frac_", "")
        
        # get the SLC value
        result = combination[(combination['landcover'] == int(land_cover_value)) & (combination['soil'] == int(soil_type_value))]['SLC']
        column_name = 'SLC_'+str(result.values[0])
        landcover_type_prepared.loc[index, column_name] = landcover_type_prepared[i].iloc[index]
        
        

landcover_type_prepared


,COMID,lat,lon,frac_1,frac_2,frac_5,frac_6,frac_8,frac_10,frac_14,...,SLC_12,SLC_13,SLC_14,SLC_15,SLC_16,SLC_17,SLC_18,SLC_19,soil,non_zero_columns
0,71022153,52.276063,-113.868966,0.006206,0.000000,0.000328,0.007333,0.062492,0.049844,0.022872,...,0.0,0.000000,0.022872,0.412466,0.000328,0.0,0.007333,0.0,3,"[frac_1, frac_5, frac_6, frac_8, frac_10, frac..."
1,71022160,52.153275,-113.963718,0.020160,0.000000,0.000401,0.009748,0.048053,0.034741,0.013718,...,0.0,0.000000,0.013718,0.773965,0.000401,0.0,0.009748,0.0,3,"[frac_1, frac_5, frac_6, frac_8, frac_10, frac..."
2,71022164,52.071025,-114.040900,0.028956,0.000000,0.000000,0.001581,0.035506,0.023226,0.028273,...,0.0,0.000000,0.028273,0.794070,0.000000,0.0,0.001581,0.0,3,"[frac_1, frac_6, frac_8, frac_10, frac_14, fra..."
3,71022165,52.062767,-114.069721,0.100058,0.000000,0.000000,0.001019,0.017309,0.049464,0.062767,...,0.0,0.000000,0.062767,0.662401,0.000000,0.0,0.001019,0.0,3,"[frac_1, frac_6, frac_8, frac_10, frac_14, fra..."
4,71022168,52.054057,-114.096759,0.032444,0.000000,0.000000,0.000000,0.018457,0.020857,0.019095,...,0.0,0.000000,0.019095,0.849063,0.000000,0.0,0.000000,0.0,3,"[frac_1, frac_8, frac_10, frac_14, frac_15, fr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,71039048,50.912529,-115.465044,0.305864,0.000000,0.000000,0.000000,0.018582,0.168456,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,3,"[frac_1, frac_8, frac_10, frac_16, frac_18, fr..."
449,71039109,50.931672,-115.554518,0.263175,0.000000,0.000000,0.000000,0.084291,0.159420,0.000643,...,0.0,0.000000,0.000643,0.000000,0.000000,0.0,0.000000,0.0,3,"[frac_1, frac_8, frac_10, frac_14, frac_16, fr..."
450,71039121,50.917342,-115.216567,0.314821,0.000743,0.000000,0.000000,0.085995,0.104014,0.000000,...,0.0,0.000743,0.000000,0.000000,0.000000,0.0,0.000000,0.0,3,"[frac_1, frac_2, frac_8, frac_10, frac_16, fra..."
451,71039250,50.952414,-115.034746,0.503254,0.000000,0.000000,0.000000,0.000133,0.095133,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,3,"[frac_1, frac_8, frac_10, frac_16, frac_17, fr..."


In [30]:
riv = gpd.read_file('../data/domain/Bow_at_Calgary_riv.shp')
riv.sort_values(by='COMID').reset_index(drop=True)
riv['lengthm'] = 0.00
riv['lengthm'] = riv['lengthkm'] * 1000

cat = gpd.read_file('../data/domain/Bow_at_Calgary_cat.shp')
cat.sort_values(by='COMID').reset_index(drop=True)
cat['area'] = 0.00
cat['area'] = cat['unitarea'] * 1000000 # km2 to m2
cat['latitude'] = cat.centroid.y
cat['longitude'] = cat.centroid.x

/var/folders/yh/b1qy7zb96k980mcb2ps9n6d9t1c6zr/T/ipykernel_18391/2469321112.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cat['latitude'] = cat.centroid.y
/var/folders/yh/b1qy7zb96k980mcb2ps9n6d9t1c6zr/T/ipykernel_18391/2469321112.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cat['longitude'] = cat.centroid.x


In [31]:
# add information to the geodata dataframe
landcover_type_prepared['NextDownID'] = riv['NextDownID']
#landcover_type_prepared['area'] = 0.00
landcover_type_prepared['area'] = cat['area']
landcover_type_prepared['latitude'] = cat['latitude']
landcover_type_prepared['longitude'] = cat['longitude']
landcover_type_prepared['elev_mean'] = elevation_mean['mean']
landcover_type_prepared['slope_mean'] = riv['slope']
landcover_type_prepared['rivlen'] = riv['lengthm']
landcover_type_prepared['uparea'] = riv['uparea']


In [32]:
column_name_mapping = {
    'COMID': 'subid',
    'NextDownID': 'maindown',
    'area': 'area',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'elev_mean': 'elev_mean',
    'slope_mean': 'slope_mean',
    'rivlen': 'rivlen'
}

# Rename the columns based on the dictionary
landcover_type_prepared = landcover_type_prepared.rename(columns=column_name_mapping)
landcover_type_prepared

#
slc_columns = [col for col in landcover_type_prepared.columns if col.startswith('SLC_')]

# Sort the columns as per your requirements
column_order = ['subid', 'maindown', 'area', 'latitude', 'longitude', 'elev_mean', 'slope_mean', 'rivlen'] + slc_columns + ['uparea']

landcover_type_prepared = landcover_type_prepared[column_order]
landcover_type_prepared

,subid,maindown,area,latitude,longitude,elev_mean,slope_mean,rivlen,SLC_1,SLC_2,...,SLC_11,SLC_12,SLC_13,SLC_14,SLC_15,SLC_16,SLC_17,SLC_18,SLC_19,uparea
0,71022153,-9999,6.964142e+07,51.988454,-114.060659,893.197144,0.000972,9650.665556,0.0,0.0,...,0.006206,0.0,0.000000,0.022872,0.412466,0.000328,0.0,0.007333,0.0,11718.725317
1,71022160,71022153,1.083574e+08,51.015383,-114.919835,915.923584,0.000848,30018.433408,0.0,0.0,...,0.020160,0.0,0.000000,0.013718,0.773965,0.000401,0.0,0.009748,0.0,11420.248116
2,71022164,71022160,4.463231e+07,50.989031,-114.860770,912.301208,0.000969,4117.764753,0.0,0.0,...,0.028956,0.0,0.000000,0.028273,0.794070,0.000000,0.0,0.001581,0.0,11158.223079
3,71022165,71022164,1.136992e+08,50.936273,-115.323424,899.861145,0.000505,1382.912645,0.0,0.0,...,0.100058,0.0,0.000000,0.062767,0.662401,0.000000,0.0,0.001019,0.0,11121.798002
4,71022168,71022165,5.303853e+07,51.965258,-113.911621,913.475586,0.000810,3820.904367,0.0,0.0,...,0.032444,0.0,0.000000,0.019095,0.849063,0.000000,0.0,0.000000,0.0,11050.683368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,71039048,71035979,2.950940e+07,51.194097,-114.572981,2255.663086,0.036308,3473.762210,0.0,0.0,...,0.305864,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,30.339439
449,71039109,71036389,7.289865e+06,51.200962,-114.506737,2300.858887,0.017588,6307.483695,0.0,0.0,...,0.263175,0.0,0.000000,0.000643,0.000000,0.000000,0.0,0.000000,0.0,41.987611
450,71039121,71035238,1.982459e+06,51.184648,-114.491796,2161.263428,0.028829,5537.016700,0.0,0.0,...,0.314821,0.0,0.000743,0.000000,0.000000,0.000000,0.0,0.000000,0.0,52.094426
451,71039250,71028304,9.345949e+07,51.156160,-114.406508,1912.747437,0.032782,2518.299361,0.0,0.0,...,0.503254,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,27.957672


In [33]:
landcover_type_prepared = landcover_type_prepared.sort_values(by='uparea').reset_index(drop=True)
landcover_type_prepared = landcover_type_prepared.drop(columns=['uparea'])
landcover_type_prepared.to_csv('../data/HYPE/GeoData.txt', sep='\t', index=False)
landcover_type_prepared

,subid,maindown,area,latitude,longitude,elev_mean,slope_mean,rivlen,SLC_1,SLC_2,...,SLC_10,SLC_11,SLC_12,SLC_13,SLC_14,SLC_15,SLC_16,SLC_17,SLC_18,SLC_19
0,71031130,71028757,3.671903e+07,51.560445,-115.920782,1357.901611,0.063497,218.630117,0.0,0.0,...,0.144020,0.170006,0.0,0.000000,0.000395,0.007215,0.000862,0.0,0.001601,0.0
1,71031429,71030308,9.854331e+05,50.286811,-114.591411,2355.891113,0.025596,327.762133,0.0,0.0,...,0.085177,0.258669,0.0,0.002177,0.000393,0.001215,0.000000,0.0,0.000179,0.0
2,71024578,71022164,2.907843e+07,51.695326,-115.378993,920.381531,0.007512,730.218286,0.0,0.0,...,0.008730,0.001135,0.0,0.000000,0.018403,0.903739,0.000177,0.0,0.000000,0.0
3,71032292,71030270,1.840258e+07,51.884675,-115.092070,2506.075439,0.013547,493.825993,0.0,0.0,...,0.069477,0.103182,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,71024619,71022248,7.864095e+07,50.654073,-115.254822,1058.150391,0.007155,1479.544192,0.0,0.0,...,0.025274,0.087812,0.0,0.000000,0.045331,0.689133,0.000000,0.0,0.003161,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,71022168,71022165,5.303853e+07,51.965258,-113.911621,913.475586,0.000810,3820.904367,0.0,0.0,...,0.020857,0.032444,0.0,0.000000,0.019095,0.849063,0.000000,0.0,0.000000,0.0
449,71022165,71022164,1.136992e+08,50.936273,-115.323424,899.861145,0.000505,1382.912645,0.0,0.0,...,0.049464,0.100058,0.0,0.000000,0.062767,0.662401,0.000000,0.0,0.001019,0.0
450,71022164,71022160,4.463231e+07,50.989031,-114.860770,912.301208,0.000969,4117.764753,0.0,0.0,...,0.023226,0.028956,0.0,0.000000,0.028273,0.794070,0.000000,0.0,0.001581,0.0
451,71022160,71022153,1.083574e+08,51.015383,-114.919835,915.923584,0.000848,30018.433408,0.0,0.0,...,0.034741,0.020160,0.0,0.000000,0.013718,0.773965,0.000401,0.0,0.009748,0.0


In [34]:
combination

combination = combination.rename(columns={'landcover': 'LULC'})
combination = combination.rename(columns={'soil': 'SOIL TYPE'})
combination = combination[['SLC','LULC','SOIL TYPE']]
combination['Main crop cropid'] = 0
combination['Second crop cropid'] = 0
combination['Crop rotation group'] = 0
combination['Vegetation type'] = 1
combination['Special class code'] = 0
combination['Tile depth'] = 0
combination['Stream depth'] = 2.296
combination['Number of soil layers'] = 3
combination['Soil layer depth 1'] = 0.091
combination['Soil layer depth 2'] = 0.493
combination['Soil layer depth 3'] = 2.296

combination

,SLC,LULC,SOIL TYPE,Main crop cropid,Second crop cropid,Crop rotation group,Vegetation type,Special class code,Tile depth,Stream depth,Number of soil layers,Soil layer depth 1,Soil layer depth 2,Soil layer depth 3
0,1,8,8,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
1,2,19,8,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
2,3,10,8,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
3,4,16,3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
4,5,1,8,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
5,6,17,3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
6,7,18,3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
7,8,8,3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
8,9,19,3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
9,10,10,3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296


In [35]:
# Add commented lines
commented_lines = [
"""! MODIS landcover													
! Add legend (raster value) and discription													
!	original legend (raster_value)	description											
!   1: 'Temperate or sub-polar needleleaf forest',
!   2: 'Sub-polar taiga needleleaf forest',
!   3: 'Tropical or sub-tropical broadleaf evergreen forest',
!   4: 'Tropical or sub-tropical broadleaf deciduous forest',
!   5: 'Temperate or sub-polar broadleaf deciduous forest',
!   6: 'Mixed forest',
!   7: 'Tropical or sub-tropical shrubland',
!   8: 'Temperate or sub-polar shrubland',
!   9: 'Tropical or sub-tropical grassland',
!   10: 'Temperate or sub-polar grassland',
!   11: 'Sub-polar or polar shrubland-lichen-moss',
!   12: 'Sub-polar or polar grassland-lichen-moss',
!   13: 'Sub-polar or polar barren-lichen-moss',
!   14: 'Wetland',
!   15: 'Cropland',
!   16: 'Barren lands',
!   17: 'Urban',
!   18: 'Water',
!   19: 'Snow and Ice',											
!													
!													
!													
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------													
!	SoilGrid V1												
!		original legend (raster_value)	description										
!	 C 	    1	 clay										
!	 SIC 	2	 silty clay										
!	 SC 	3	 sandy clay										
!	 CL 	4	 clay loam										
!	 SICL 	5	 silty clay loam										
!	 SCL 	6	 sandy clay loam										
!	 L   	7	 loam										
!	 SIL 	8	 silty loam										
!	 SL 	9	 sandy loam										
!	 SI 	10	 silt										
!	 LS 	11	 loamy sand										
!	 S  	12	 sand										
!													
!													
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	
!          SLC	LULC	SOIL TYPE	Main crop cropid	Second crop cropid	Crop rotation group	Vegetation type	Special class code	Tile depth	Stream depth	Number of soil layers	Soil layer depth 1	Soil layer depth 2	Soil layer depth 3"""
]

# Open the file in write mode
with open('../data/HYPE/GeoClass.txt', 'w') as file:
    # Write the commented lines
    for line in commented_lines:
        file.write(line + '\n')

# writing the `GeoClass.txt` file
with open('../data/HYPE/GeoClass.txt', 'a') as file:
        combination.to_csv(file, sep='\t', index=False, header=False)

In [36]:
# create a smaple forcing file
import datetime

# List of column names
columns = list (landcover_type_prepared['subid'])

# Start date and end date
start_date = datetime.date(2023, 10, 1)
end_date = start_date + datetime.timedelta(days=9)  # 10 days

# List of dates for 10 days
dates = [start_date + datetime.timedelta(days=i) for i in range(10)]

# Create an empty DataFrame
df = pd.DataFrame(columns=columns, index=dates)

df = df.round(3)

df.index.name = 'Date'




In [37]:
for col in columns:
    df[col] = np.random.uniform(0, 1, size=len(dates))
df.to_csv('../data/HYPE/Pobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.2f')

for i, col in enumerate(columns):
    # Generate values from -5 to +5 for each column
    df[col] = [(i % 11) - 5 for i in range(len(dates))]
df.to_csv('../data/HYPE/Tobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.1f')

for col in columns:
    df[col] = df[col]+1
df.to_csv('../data/HYPE/TMAXobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.1f')

for col in columns:
    df[col] = df[col]-2
df.to_csv('../data/HYPE/TMINobs.txt', sep='\t', na_rep='', index_label='Date', float_format='%.1f')

,71027942,71027957,71027962,71027963,71027969,71027979,71027981,71027992,71028007,71028014,...,71038844,71038890,71038901,71038946,71038961,71039048,71039109,71039121,71039250,71039266
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-01,0.685898,0.758986,0.522744,0.575416,0.051444,0.916739,0.229523,0.690251,0.883905,0.258088,...,0.521634,0.818949,0.389091,0.355143,0.967483,0.568549,0.909287,0.755196,0.138123,0.730858
2023-10-02,0.235409,0.676245,0.316251,0.272614,0.238148,0.151266,0.239834,0.840245,0.029773,0.969516,...,0.552049,0.005690,0.897502,0.580055,0.824307,0.640799,0.054121,0.835454,0.699113,0.672954
2023-10-03,0.098136,0.931442,0.751397,0.981644,0.153017,0.455225,0.980516,0.826923,0.926254,0.101222,...,0.273581,0.434336,0.738575,0.132192,0.790694,0.825248,0.714215,0.647128,0.780165,0.756245
2023-10-04,0.947074,0.441704,0.410678,0.226992,0.224621,0.978368,0.306233,0.639026,0.488671,0.616723,...,0.626331,0.292437,0.344202,0.390202,0.591072,0.270164,0.178557,0.448879,0.412073,0.432565
2023-10-05,0.281160,0.475741,0.858635,0.416300,0.819710,0.853619,0.151872,0.711520,0.468179,0.494889,...,0.716843,0.565295,0.282241,0.720945,0.613645,0.117075,0.511781,0.159996,0.183635,0.648685
2023-10-06,0.002077,0.154730,0.161290,0.728251,0.197299,0.517063,0.334675,0.505035,0.106449,0.397127,...,0.577354,0.473251,0.262803,0.370551,0.460292,0.594687,0.790258,0.299101,0.616819,0.598528
2023-10-07,0.903236,0.888245,0.400290,0.283269,0.427782,0.876105,0.627176,0.058576,0.239038,0.347561,...,0.856837,0.243458,0.033217,0.457467,0.895701,0.096847,0.738913,0.086411,0.686420,0.779089
2023-10-08,0.424316,0.943395,0.915103,0.600148,0.375364,0.488899,0.563745,0.402680,0.794224,0.115797,...,0.831211,0.470361,0.555253,0.409439,0.079403,0.476496,0.365252,0.770439,0.551365,0.166691
2023-10-09,0.115455,0.242523,0.051540,0.370798,0.854388,0.615938,0.092642,0.008170,0.873188,0.205594,...,0.861138,0.959750,0.562908,0.751206,0.347275,0.781608,0.404012,0.414536,0.930569,0.570845


In [48]:
riv = gpd.read_file('../data/domain/Bow_at_Calgary_riv.shp')
riv

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,geometry
0,71027942,28.145110,20.588731,1.367015,0.001798,7857.020581,4,50.7,0.001798,-9999,2,71027957,71030749,0,0,"LINESTRING (-114.04167 51.04583, -114.04250 51..."
1,71027957,20.589551,13.558710,1.518548,0.001508,7669.065145,4,31.1,0.001508,71027942,2,71027962,71030189,0,0,"LINESTRING (-114.30750 51.12583, -114.30833 51..."
2,71027962,2.160029,1.774091,1.217541,0.002403,7406.819260,4,5.2,0.002403,71027957,2,71027963,71028062,0,0,"LINESTRING (-114.48167 51.18000, -114.48250 51..."
3,71027963,1.951061,1.625592,1.200215,0.001484,6803.660756,4,2.9,0.001484,71027962,2,71027969,71030314,0,0,"LINESTRING (-114.50250 51.18917, -114.50333 51..."
4,71027969,4.507794,3.805159,1.184653,0.002480,6713.480813,4,11.2,0.002480,71027963,2,71027979,71031431,0,0,"LINESTRING (-114.52167 51.19750, -114.52250 51..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,71039048,3.473762,2.989757,1.161888,0.036308,30.339439,1,126.3,0.036308,71035979,0,0,0,0,0,"LINESTRING (-115.41583 50.87417, -115.41667 50..."
165,71039109,6.307484,4.937584,1.277443,0.017588,41.987611,1,111.1,0.017588,71036389,0,0,0,0,0,"LINESTRING (-115.51417 50.88583, -115.51500 50..."
166,71039121,5.537017,4.428210,1.250396,0.028829,52.094426,1,160.0,0.028829,71035238,0,0,0,0,0,"LINESTRING (-115.13083 50.93000, -115.13167 50..."
167,71039250,2.518299,2.040736,1.234015,0.032782,27.957672,1,82.7,0.032782,71028304,0,0,0,0,0,"LINESTRING (-115.09500 50.98083, -115.09417 50..."


In [39]:
landcover_type_prepared['maindown'].values

array([71028757, 71030308, 71022164, 71030270, 71022248, 71030390,
       71035360, 71030314, 71024292, 71029759, 71035930, 71030100,
       71028681, 71036085, 71034975, 71035946, 71030287, 71028304,
       71028722, 71028961, 71029212, 71022754, 71028699, 71035105,
       71023982, 71028867, 71024386, 71035979, 71028957, 71036098,
       71029016, 71028795, 71027979, 71024082, 71035895, 71030071,
       71029879, 71029857, 71028249, 71036422, 71035810, 71028054,
       71030292, 71030189, 71036466, 71028023, 71023994, 71029380,
       71036211, 71030243, 71030265, 71036250, 71029715, 71024198,
       71028734, 71029766, 71027942, 71022209, 71029646, 71024386,
       71028663, 71030302, 71028597, 71030177, 71029963, 71036170,
       71029803, 71030390, 71024292, 71029721, 71022426, 71028974,
       71023889, 71030156, 71030313, 71035337, 71036422, 71030314,
       71029158, 71024229, 71022911, 71036389, 71024229, 71022365,
       71030057, 71022305, 71029150, 71036466, 71028675, 71036